## MLOps am Beispiel von MLFlow

see: https://mlflow.org/docs/latest/ml/ 

MLflow ist eine Open-Source-Plattform, die speziell entwickelt wurde, um Machine-Learning-Anwender und -Teams bei der Bewältigung der Komplexität des Machine-Learning-Prozesses zu unterstützen. MLflow konzentriert sich auf den gesamten Lebenszyklus von Machine-Learning-Projekten und stellt sicher, dass jede Phase verwaltbar, nachvollziehbar und reproduzierbar ist.

Diese Übung baut auf folgendem Tutorial auf: https://mlflow.org/docs/latest/ml/getting-started/quickstart/

Sie vermuten bereits: ja, wir benötigen mal wieder eine neue Library :). Installieren Sie mit pip mlflow:

In [ ]:
#TODO install mlflow 

### Starten des ML-Flow Servers

MLFlow ist eine serverbasierte Anwendung. Sie können somit das Backend entweder in der Cloud oder lokal auf Ihrem eigenen Rechner laufen lassen. Der Einfachheit halber läuft im folgenden der MLFlow Server auf Ihrem lokalen Rechner. Sofern Sie mit mehreren Entwicklern an einem Machine Learning Projekt arbeiten, ist es sicherlich hilfreich zu einer Cloud-Lösung zu greifen.

Gut, nun müssen wir zuerst den MLFlow Server starten. Dies geht leider nicht aus dem Jupyter Notebook, führen Sie daher folgende Schritte mit dem Terminal oder Powershell aus:

1) Aktivieren Sie Ihr virtuelles Environment:

Mac: ```source .venv/bin/activate```

Windows: ```.venv/Scripts/activate.ps1```

2) Starten Sie den MLFlow Server:

```mlflow server --backend-store-uri sqlite:///mlflow.db --port 5000```

3) Öffnen Sie die UI des MLFlow Servers im Browser:

http://127.0.0.1:5000 

### Vorbereitung: Datensatz und Modell

Wir möchten anhand des IRIS-Datensatzes einen einfachen Klassifikator mit 3 Klassen trainieren. Der Iris-Datensatz ist einer der bekanntesten und am häufigsten verwendeten Datensätze in der Statistik, im maschinellen Lernen und in der Datenanalyse.

**Ursprung:** Der Datensatz wurde 1936 von dem britischen Statistiker und Biologen Ronald A. Fisher in seiner Arbeit zur diskriminanzanalytischen Methode veröffentlicht.

**Zweck:** Fisher nutzte die Daten, um zu zeigen, wie man anhand von morphologischen Merkmalen verschiedene Arten der Gattung Iris (Schwertlilien) unterscheiden kann.

Der Iris-Datensatz enthält 150 Einträge (Proben) von drei Arten der Gattung Iris: Iris setosa, Iris versicolor, Iris virginica

Für jede Probe wurden vier Merkmale gemessen (in Zentimetern):

| Merkmal | Beschreibung |
| ------- | ------------ |
|Sepal Length|Länge des Kelchblatts|
|Sepal Width|Breite des Kelchblatts|
|Petal Length|Länge des Blütenblatts|
|Petal Width|Breite des Blütenblatts|

Wir bereiten den Datensatz vor:

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Über ein dictionary definieren wir die Parameter für das Modelltraining:

In [ ]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8877,
}

### Autologging mit MLflow

Im folgenden trainieren wir eine einfache LogisticRegression als Klassifikator. Mit nur einer Zeile können wir MLFlow dazu bringen, das Experiemnt zu tracken. Schauen Sie im https://mlflow.org/docs/latest/ml/getting-started/quickstart/ nach und fügen die entsprechende Zeile hinzu:

In [ ]:
import mlflow

#TODO: set mlflow parameters and autologging 

# Just train the model normally
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

Schauen Sie sich in der Weboberfläche an, was MLflow für Sie geloggt hat. Neben sämtlichen Metriken werden auch der Datensatz und das Modell geloggt. Suchen Sie die confusion matrix!

## Manuelles Logging mit MLflow

Nachdem wir nun gelernt haben, wie man einen Modelltrainingslauf mit MLflow-Autologging protokolliert, wollen wir einen Schritt weiter gehen und lernen, wie man ein Modell und Metadaten manuell protokolliert. Dies ist nützlich, wenn Sie mehr Kontrolle über den Protokollierungsprozess haben möchten.

Die Schritte, die wir durchführen werden, sind:

1) Starten Sie einen MLflow-Laufkontext, um einen neuen Lauf zu starten, in dem wir das Modell und die Metadaten protokollieren werden.
2) Trainieren und testen des Modells
3) Protokollieren Sie Modellparameter und Leistungsmetriken.
4) Kennzeichnen Sie den Lauf, um ihn leicht wiederzufinden.

In [ ]:
# Start an MLflow run
#TODO log with mlflow manually
    # Log the parameters
    mlflow.log_params(params)

    # Train the model
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Log the model
    mlflow.sklearn.log_model(sk_model=lr, name="iris_model", registered_model_name="iris_model_registered")

    # Predict on the test set, compute and log the loss metric
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Optional: Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

Schauen Sie sich wieder die geloggten Werte an. Was sind die Unterschiede?

## Nutzen von gespeicherten Modellen

Sofern wir ein Modell registriert haben (entweder durch registered_model_name oder in der Bedienoberfläche), können wir es einfach wieder laden:

In [ ]:
model_name = "iris_model_registered"
model_uri = f"models:/{model_name}/1" # Version 1 of the model
loaded_modell = mlflow.pyfunc.load_model(model_uri)

Jetzt haben wir unser Modell über mlflow geladen. Wir können das nun für Batch-Processing direkt in Python-Code nutzen, mit neuen Daten weiter trainieren oder ähnlich zu 4-E-HostingInterfaces über einen REST-Endpunkt zur Verfügung stellen. Als interessante Aufgabe können Sie versuchen mit einem Flask-Server das geladene Model zu hosten - nutzen Sie gerne dafür die Hilfe von einem LLM Ihrer Wahl.